In [1]:
from wayne_utils import load_data, save_data
from llm_inferencer import Register, Inferencer
import os
import sys
from tqdm import tqdm

_ROOT_PATH = "/home/jiangpeiwen2/jiangpeiwen2/T-Tuple-T"

test_path = os.path.join( _ROOT_PATH, "data/test.json")
train_path = os.path.join( _ROOT_PATH, "data/train.json")

test_list = load_data( test_path, "json")           # 754
train_list = load_data( train_path, "json")         # 3017


/home/jiangpeiwen2/jiangpeiwen2/miniconda3/envs/vllm/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-12-10 01:55:20,622	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


ERROR 12-10 01:55:21 pynccl.py:53] Failed to load NCCL library from libnccl.so.2 .It is expected if you are not running on NVIDIA/AMD GPUs.Otherwise please set the environment variable VLLM_NCCL_SO_PATH to point to the correct nccl library path.
INFO 12-10 01:55:21 pynccl_utils.py:17] Failed to import NCCL library: libnccl.so.2: cannot open shared object file: No such file or directory
INFO 12-10 01:55:21 pynccl_utils.py:18] It is expected if you are not running on NVIDIA GPUs.


In [ ]:
'''Server'''
kwargs = {}
kwargs["local_or_remote"] = "local"
kwargs["server_or_reader"] = "server"
kwargs["model_name"] = "Mistral-7B-Instruct-v0.2"
kwargs["gpu_list"] = "0"
kwargs["local_engine"] = "vllm"
inferencer = Inferencer( kwargs )


## 提示词指令

In [2]:
general_instruction = """According to the live text, please count the number of: 1.goals, 2. shots, 3.fouls, 4.yellow cards, 5.red cards, 6.corner kicks, 7.free kicks, and 8.offsides for each team. Note that goals and saved attempts and blocked attempts and missed attempts are considered shots. Handball and dangerous play are also considered foul. The second yellow card is also considered a red card. Penalty is also considered as free kicks."""

stage_1_instruction = """{INSTRUCTION}
Please extract all the relevant event from the following passage, output them in (player name, team name, event) or (team name, event) format. Constrain the event names to only the following options: 1.goals, 2.shots, 3.fouls, 4.yellow cards, 5.red cards, 6.corner kicks, 7.free kicks, and 8.offsides:
{Text}"""
stage_2_DE_instruction = """{INSTRUCTION}
Please count all the information required and integrate these tuples:
{Text}"""

stage_2_CG_instruction = """{INSTRUCTION}
Please develop a Python code to consolidate these tuples as specified:
{Text}"""
stage_3_instruction = """{INSTRUCTION}
Let’s do the following things: 
1. Extract all the relevant events from the following passage in (player name, team name, event) or (team name, event) format. 
2. Integrate these tuples. 
3. Output a table with 2 rows in CSV format.
{Text}"""



# 获取第一轮输出

In [ ]:
import re
from typing import List, Tuple

# 从字符串中抽取三元组和二元组
def unlegal_filter( tup: tuple):
    flag = 0
    ban_list = ["team name", "Team name",]
    for item in ban_list:
        if item in tup:
            flag = 1
            break
    if "Player " in tup[0]:
        flag = 1
    if flag == 0:
        return tup
    else:
        return None



def extract_pairs_and_triples(text: str) -> List[Tuple[str, ...]]:
    """
    Extract all pairs in the format (xxx, xxx) and triples in the format (xxx, xxx, xxx)
    where xxx consists of 1-2 words.

    Args:
        text (str): Input string containing the pairs and triples.

    Returns:
        List[Tuple[str, ...]]: List of extracted pairs and triples.
    """
    # Regular expressions to match pairs and triples
    triple_pattern = re.compile(r"\((\b\w+(?: \w+)?), (\b\w+(?: \w+)?), (\b\w+(?: \w+)?)\)")
    pair_pattern = re.compile(r"\((\b\w+(?: \w+)?), (\b\w+(?: \w+)?)\)")
    trip_matches = triple_pattern.findall(text)
    tuple_matches = pair_pattern.findall(text)

    ret = []
    for matches in [trip_matches, tuple_matches]:
        for match in matches:
            filt = unlegal_filter( match )
            if filt != None:
                ret.append( match )
    return ret

# Example usage
'''sample_text = 'Here are the extracted events in the format of (player name, team name, event) or (team name, event):\n\n1. (Player27, Away Team, shot)\n2. (Player27, Away Team, shot)\n3. (Away Team, corner kick)\n4. (Away Team, offside)\n5. (Away Team, corner kick)\n6. (Player29, Away Team, shot)\n7. (Player28, Away Team, free kick)\n8. (Player20, Away Team, foul)\n9. (Player7, Home Team, shot)\n10. (Player11,'
results = extract_pairs_and_triples(sample_text)
print(results)'''


"sample_text = 'Here are the extracted events in the format of (player name, team name, event) or (team name, event):\n\n1. (Player27, Away Team, shot)\n2. (Player27, Away Team, shot)\n3. (Away Team, corner kick)\n4. (Away Team, offside)\n5. (Away Team, corner kick)\n6. (Player29, Away Team, shot)\n7. (Player28, Away Team, free kick)\n8. (Player20, Away Team, foul)\n9. (Player7, Home Team, shot)\n10. (Player11,'\nresults = extract_pairs_and_triples(sample_text)\nprint(results)"

In [5]:
prompt_1_path = os.path.join( _ROOT_PATH, "prompt_1_list.pickle")
prompt_1_list = []
for i in range( len(test_list) ):
    prompt_1 = stage_1_instruction.format( INSTRUCTION = general_instruction,  Text = test_list[i]["text"])
    prompt_1_list.append( [prompt_1 ])
save_data( prompt_1_list, prompt_1_path)

In [6]:
kwargs = {}
prompt_list_from_path = prompt_1_path
kwargs["local_or_remote"] = "local"
kwargs["server_or_reader"] = "reader"
kwargs["model_name"] = "Mistral-7B-Instruct-v0.2"
kwargs["gpu_list"] = "0,1,2,3,5"
kwargs["local_engine"] = "vllm"
if prompt_list_from_path != None:
    kwargs["prompt_list_from_path"] = prompt_list_from_path
    kwargs["predict_list_to_path"] = prompt_list_from_path.replace( "prompt_1_list.pickle", "predict_1_list.pickle")
    kwargs["sample_little"] = None
inferencer = Inferencer( kwargs )

Enginevllm exists, run with vllm


WARNING 12-09 12:44:08 config.py:748] Casting torch.bfloat16 to torch.float16.
INFO 12-09 12:44:08 llm_engine.py:75] Initializing an LLM engine (v0.4.0) with config: model='/home/jiangpeiwen2/jiangpeiwen2/workspace/LLMs/Mistral-7B-Instruct-v0.2', tokenizer='/home/jiangpeiwen2/jiangpeiwen2/workspace/LLMs/Mistral-7B-Instruct-v0.2', tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.float16, max_seq_len=8192, download_dir=None, load_format=auto, tensor_parallel_size=1, disable_custom_all_reduce=True, quantization=None, enforce_eager=False, kv_cache_dtype=auto, device_config=cuda, seed=32)
WARNING 12-09 12:44:08 config.py:748] Casting torch.bfloat16 to torch.float16.
INFO 12-09 12:44:08 llm_engine.py:75] Initializing an LLM engine (v0.4.0) with config: model='/home/jiangpeiwen2/jiangpeiwen2/workspace/LLMs/Mistral-7B-Instruct-v0.2', tokenizer='/home/jiangpeiwen2/jiangpeiwen2/workspace/LLMs/Mistral-7B-Instruct-v0.2', tokenizer_mode=auto, revision

In [ ]:
# 过滤输出
predict_1_list = load_data( os.path.join(_ROOT_PATH, "predict_1_list.pickle"), "pickle")

tuples_list = []

points_dict = {
    'shot': [ 'shot', "missed", "header", "hits the bar", "goes high", "goal", "blocked", "saved"], 
    'foul':  [ 'foul', "dangerous play", "handball"], 
    'yellow card':  ['yellow card'], 
    'red card': [ "red card", "sent off"], 
    'corner kick' : [ "corner "], 
    'free kick': [ "free ", "penalty"], 
    'offside':  [ 'offside' ],
}

for i in range( len(predict_1_list) ):
    tuples = extract_pairs_and_triples(predict_1_list[i][0])
    temp = []
    for _tuple in tuples:
        # 遍历每个元组
        flag = 0
        for item in _tuple:
            # 遍历元组中每个项
            if "home " in item.lower() or "away " in item.lower():
                flag = 1
                break
        if flag == 0:
            continue
        else:
            _field = _tuple[-1]
            for key in points_dict.keys():
                if _field.lower() in key:
                    flag = 2
            if flag == 2:
                temp.append( _tuple )
    
    tuples_list.append( temp )

save_data( tuples_list, os.path.join(_ROOT_PATH, "predict_1_filter_list.pickle"))

# 获取第二轮输出

In [3]:
tuples_list = load_data( os.path.join(_ROOT_PATH, "predict_1_filter_list.pickle"), "pickle")

In [4]:
prompt_2_path = os.path.join(_ROOT_PATH, "prompt_2_list.pickle" )

In [5]:
valid_to_orignal = []
valid_list = []
for i in range( len(tuples_list) ):
    tuples = tuples_list[i]
    if len(tuples)>0:
        prompt_2 = stage_2_CG_instruction.format( INSTRUCTION = general_instruction,  Text = f"{tuples}" )
        valid_list.append( [prompt_2] )
        valid_to_orignal.append(i )

save_data( valid_list, prompt_2_path )
save_data( valid_to_orignal, os.path.join(_ROOT_PATH, "prompt_2_valid_to_orignal.pickle" ) )

In [4]:
valid_list = load_data( prompt_2_path, "pickle" )

In [6]:
kwargs = {}
prompt_list_from_path = prompt_2_path  # prompt_2_merge_path
kwargs["local_or_remote"] = "local"
kwargs["server_or_reader"] = "reader"
kwargs["model_name"] = "Mistral-7B-Instruct-v0.2"
kwargs["gpu_list"] = "0,1,2,3,4,5"
kwargs["local_engine"] = "vllm"
if prompt_list_from_path != None:
    kwargs["prompt_list_from_path"] = prompt_list_from_path
    kwargs["predict_list_to_path"] = prompt_list_from_path.replace( "prompt_2_list.pickle", "predict_2_list.pickle")
    kwargs["sample_little"] = None
inferencer = Inferencer( kwargs )

Enginevllm exists, run with vllm
WARNING 12-10 01:55:28 config.py:748] Casting torch.bfloat16 to torch.float16.
INFO 12-10 01:55:28 llm_engine.py:75] Initializing an LLM engine (v0.4.0) with config: model='/home/jiangpeiwen2/jiangpeiwen2/workspace/LLMs/Mistral-7B-Instruct-v0.2', tokenizer='/home/jiangpeiwen2/jiangpeiwen2/workspace/LLMs/Mistral-7B-Instruct-v0.2', tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.float16, max_seq_len=8192, download_dir=None, load_format=auto, tensor_parallel_size=1, disable_custom_all_reduce=True, quantization=None, enforce_eager=False, kv_cache_dtype=auto, device_config=cuda, seed=32)
WARNING 12-10 01:55:28 config.py:748] Casting torch.bfloat16 to torch.float16.
INFO 12-10 01:55:28 llm_engine.py:75] Initializing an LLM engine (v0.4.0) with config: model='/home/jiangpeiwen2/jiangpeiwen2/workspace/LLMs/Mistral-7B-Instruct-v0.2', tokenizer='/home/jiangpeiwen2/jiangpeiwen2/workspace/LLMs/Mistral-7B-Instruct-v0.

# 获取第三轮输出

In [7]:
predict_2_list = load_data( os.path.join(_ROOT_PATH, "predict_2_list.pickle"), "pickle")

In [29]:
executable_list = []

for i in range( len(predict_2_list) ):
    text = predict_2_list[i][0].split("```python")[1].strip()
    text = text.split("```")[0].strip()
    try:
        exec( text )
        executable_list.append( True )
    except:
        executable_list.append( False )
        print( f"{i} Failed" )

0 Failed
1 Failed
{'goals': 0, 'shots': 0, 'fouls': 11, 'yellow_cards': 3, 'red_cards': 0, 'corner_kicks': 0, 'free_kicks': 0, 'offsides': 2}
{'goals': 0, 'shots': 0, 'fouls': 7, 'yellow_cards': 2, 'red_cards': 0, 'corner_kicks': 0, 'free_kicks': 0, 'offsides': 1}
{'goals': 0, 'shots': 0, 'fouls': 6, 'yellow_cards': 2, 'red_cards': 0, 'corner_kicks': 0, 'offsides': 2}
{'goals': 0, 'shots': 1, 'fouls': 4, 'yellow_cards': 2, 'red_cards': 0, 'corner_kicks': 1, 'offsides': 1}
4 Failed
5 Failed
6 Failed
7 Failed
8 Failed
9 Failed
10 Failed
11 Failed
{'goals': 2, 'shots': 15, 'fouls': 11, 'yellow_cards': 1, 'red_cards': 0, 'corner_kicks': 6, 'free_kicks': 11}
{'goals': 0, 'shots': 15, 'fouls': 11, 'yellow_cards': 2, 'red_cards': 1, 'corner_kicks': 0, 'free_kicks': 11}
13 Failed
14 Failed
15 Failed
16 Failed
17 Failed
18 Failed
19 Failed
{'Away': {'goals': 0, 'shots': 0, 'fouls': 0, 'yellow_cards': 0, 'red_cards': 0, 'corner_kicks': 0, 'free_kicks': 0}, 'Home': {'goals': 0, 'shots': 0, 'fouls

In [31]:
excuta = {
    2: {
        "home team": {'goals': 0, 'shots': 0, 'fouls': 11, 'yellow_cards': 3, 'red_cards': 0, 'corner_kicks': 0, 'free_kicks': 0, 'offsides': 2},
        "away team": {'goals': 0, 'shots': 0, 'fouls': 7, 'yellow_cards': 2, 'red_cards': 0, 'corner_kicks': 0, 'free_kicks': 0, 'offsides': 1}
    },
    3: {
        "home team": {'goals': 0, 'shots': 0, 'fouls': 6, 'yellow_cards': 2, 'red_cards': 0, 'corner_kicks': 0, 'offsides': 2},
        "away team": {'goals': 0, 'shots': 1, 'fouls': 4, 'yellow_cards': 2, 'red_cards': 0, 'corner_kicks': 1, 'offsides': 1}
    },
    12: {
        "home team": {'goals': 2, 'shots': 15, 'fouls': 11, 'yellow_cards': 1, 'red_cards': 0, 'corner_kicks': 6, 'free_kicks': 11},
        "away team": {'goals': 0, 'shots': 15, 'fouls': 11, 'yellow_cards': 2, 'red_cards': 1, 'corner_kicks': 0, 'free_kicks': 11}
    },
    20: {
        'Away': {'goals': 0, 'shots': 0, 'fouls': 0, 'yellow_cards': 0, 'red_cards': 0, 'corner_kicks': 0, 'free_kicks': 0}, 
        'Home': {'goals': 0, 'shots': 0, 'fouls': 0, 'yellow_cards': 0, 'red_cards': 0, 'corner_kicks': 0, 'free_kicks': 0}, 
        'free': {'goals': 0, 'shots': 0, 'fouls': 0, 'yellow_cards': 0, 'red_cards': 0, 'corner_kicks': 0, 'free_kicks': 0}, 
        'corner': {'goals': 0, 'shots': 0, 'fouls': 0, 'yellow_cards': 0, 'red_cards': 0, 'corner_kicks': 0, 'free_kicks': 0}
    },
    21: {
        "home team": {'Goals': 0, 'Shots': 0, 'Fouls': 2, 'Yellow Cards': 1, 'Red Cards': 1, 'Corner_Kicks': 0, 'Offsides': 1},
        "away team": {'Goals': 0, 'Shots': 0, 'Fouls': 8, 'Yellow Cards': 0, 'Red Cards': 4, 'Corner_Kicks': 2, 'Offsides': 0}
    },
    27: {'Home Team': {'goals': 0, 'shots': 0, 'fouls': 0, 'yellow_cards': 0, 'red_cards': 0, 'corner_kicks': 0, 'offsides': 0}, 'Away Team': {'goals': 0, 'shots': 0, 'fouls': 0, 'yellow_cards': 0, 'red_cards': 0, 'corner_kicks': 0, 'offsides': 0}},
    29: {
        "home team": {'goals': 0, 'shots': 13, 'fouls': 14, 'yellow_cards': 8, 'red_cards': 0, 'corner_kicks': 0, 'offsides': 0},
        "away team": {'goals': 0, 'shots': 9, 'fouls': 13, 'yellow_cards': 6, 'red_cards': 1, 'corner_kicks': 0, 'offsides': 0}
    },
    31: {
        "home team": """Goals: 1
                    Shots: 6
                    Fouls: 6
                    Yellow Cards: 2
                    Red Cards: 4
                    Corner Kicks: 3
                    Free Kicks: 0
                    Offsides: 0""",
        "away team": """"Goals: 6
                    Shots: 7
                    Fouls: 7
                    Yellow Cards: 2
                    Red Cards: 5
                    Corner Kicks: 2
                    Free Kicks: 0
                    Offsides: 0"""
    },
    40: {
        "home team": {'goals': 0, 'shots': 0, 'fouls': 0, 'yellow_cards': 0, 'red_cards': 0, 'corner_kicks': 0, 'free_kicks': 0, 'offsides': 0},
        "away team": {'goals': 0, 'shots': 0, 'fouls': 0, 'yellow_cards': 0, 'red_cards': 0, 'corner_kicks': 0, 'free_kicks': 0, 'offsides': 0}
    },
    41: {
        "home team": {'goals': 0, 'shots': 1, 'fouls': 9, 'yellow_cards': 1, 'red_cards': 1, 'corner_kicks': 0, 'free_kicks': 4, 'offsides': 2},
        "away team": {'goals': 0, 'shots': 5, 'fouls': 3, 'yellow_cards': 3, 'red_cards': 1, 'corner_kicks': 0, 'free_kicks': 9, 'offsides': 0}
    },
    42: {
        "home team": {'goals': 0, 'shots': 28, 'fouls': 13, 'yellow_cards': 0, 'red_cards': 0, 'corner_kicks': 0, 'offsides': 0},
        "away team": {'goals': 0, 'shots': 3, 'fouls': 11, 'yellow_cards': 4, 'red_cards': 3, 'corner_kicks': 0, 'offsides': 2}
    },
    48: {
        "home team": {'Goals': 0, 'Shots': 0, 'Fouls': 5, 'Yellow cards': 0, 'Red cards': 0, 'Corner_kicks': 3, 'Free_kicks': 7, 'Offsides': 0},
        "away team": {'Goals': 0, 'Shots': 2, 'Fouls': 9, 'Yellow cards': 2, 'Red cards': 0, 'Corner_kicks': 0, 'Free_kicks': 6, 'Offsides': 2}
    },
    51: {
        "home team": {'goals': 0, 'shots': 7, 'fouls': 9, 'yellow_cards': 2, 'red_cards': 7, 'corner_kicks': 0, 'free_kicks': 0, 'offsides': 0},
        "away team": {'goals': 0, 'shots': 4, 'fouls': 3, 'yellow_cards': 2, 'red_cards': 1, 'corner_kicks': 0, 'free_kicks': 0, 'offsides': 1}
    },
    54: {
        "home team": {'goals': 0, 'shots': 0, 'fouls': 0, 'yellow_cards': 0, 'red_cards': 0, 'corner_kicks': 0, 'offsides': 0},
        "away team": {'goals': 0, 'shots': 0, 'fouls': 0, 'yellow_cards': 0, 'red_cards': 0, 'corner_kicks': 0, 'offsides': 0}
    },
    56: {
        "home team": """Home Team Stats:
                        Goals: 0
                        Shots: 0
                        Fouls: 7
                        Yellow Cards: 1
                        Red Cards: 0
                        Corner Kicks: 0
                        Free Kicks: 3
                        Offsides: 4
                        """,
        "away team": """Away Team Stats:
                        Goals: 0
                        Shots: 0
                        Fouls: 5
                        Yellow Cards: 5
                        Red Cards: 4
                        Corner Kicks: 0
                        Free Kicks: 7
                        Offsides: 2"""
    },
    59: {
        "home team": "Home Team Stats: Goals=0, Shots=2, Fouls=2, Yellow Cards=1, Red Cards=0, Corner Kicks=0, Free Kicks=2, Offsides=2",
        "away team": "Away Team Stats: Goals=0, Shots=11, Fouls=6, Yellow Cards=1, Red Cards=0, Corner Kicks=0, Free Kicks=2, Offsides=2"
    },
    60: {
        "home team": """Home Team Stats:
                        Goals: 0
                        Shots: 6
                        Fouls: 5
                        Yellow Cards: 1
                        Red Cards: 0
                        Corner Kicks: 0
                        Offsides: 5""",
        "away team": """Away Team Stats:
                        Goals: 0
                        Shots: 11
                        Fouls: 3
                        Yellow Cards: 1
                        Red Cards: 0
                        Corner Kicks: 0
                        Offsides: 1"""
    },
    62: {
        "home team": {'goals': 0, 'shots': 0, 'fouls': 2, 'yellow_cards': 2, 'red_cards': 0, 'corner_kicks': 0, 'free_kicks': 0, 'offsides': 0},
        "away team": {'goals': 0, 'shots': 0, 'fouls': 6, 'yellow_cards': 2, 'red_cards': 4, 'corner_kicks': 0, 'free_kicks': 5, 'offsides': 3}
    },
    63: {
        "home team": """Goals: 0
                        Shots: 7
                        Fouls: 6
                        Yellow Cards: 2
                        Red Cards: 4
                        Corner Kicks: 0
                        Free Kicks: 4
                        Offsides: 0""",
        "away team": """Goals: 0
                        Shots: 0
                        Fouls: 4
                        Yellow Cards: 2
                        Red Cards: 2
                        Corner Kicks: 0
                        Free Kicks: 1
                        Offsides: 0"""
    },
    68: {'Home Team': {'goals': 0, 'shots': 0, 'fouls': 0, 'yellow_cards': 0, 'red_cards': 0, 'corner_kicks': 0, 'offsides': 0}, 'Away Team': {'goals': 0, 'shots': 0, 'fouls': 0, 'yellow_cards': 0, 'red_cards': 0, 'corner_kicks': 0, 'offsides': 0}},
    92: {
        "home team": {'Goals': 0, 'Shots': 8, 'Fouls': 3, 'Yellow Cards': 1, 'Red Cards': 0, 'Corner Kicks': 0, 'Offsides': 1},
        "away team": {'Goals': 0, 'Shots': 0, 'Fouls': 9, 'Yellow Cards': 0, 'Red Cards': 0, 'Corner Kicks': 0, 'Offsides': 2}
    },
    105: {
        "home team": {'Goals': 0, 'Shots': 4, 'Fouls': 7, 'Yellow Cards': 1, 'Red Cards': 0, 'Corner Kicks': 1, 'Offsides': 2},
        "away team": {'Goals': 0, 'Shots': 8, 'Fouls': 2, 'Yellow Cards': 1, 'Red Cards': 0, 'Corner Kicks': 2, 'Offsides': 0}
    },
    111: {
        "home team": {'goals': 0, 'shots': 0, 'fouls': 7, 'yellow_cards': 1, 'red_cards': 0, 'corner_kicks': 1, 'offsides': 0},
        "away team": {'goals': 0, 'shots': 0, 'fouls': 7, 'yellow_cards': 3, 'red_cards': 1, 'corner_kicks': 3, 'offsides': 4}
    },
    114: {
        "home team": {'goals': 0, 'shots': 0, 'fouls': 5, 'yellow_cards': 2, 'red_cards': 3, 'corner_kicks': 0, 'free_kicks': 6, 'offsides': 1},
        "away team": {'goals': 0, 'shots': 3, 'fouls': 2, 'yellow_cards': 2, 'red_cards': 0, 'corner_kicks': 0, 'free_kicks': 5, 'offsides': 1}
    },
    128: {
        "home team": {'goals': 0, 'shots': 3, 'fouls': 10, 'yellow_cards': 2, 'red_cards': 0, 'corner_kicks': 0, 'free_kicks': 3},
        "away team": {'goals': 0, 'shots': 8, 'fouls': 4, 'yellow_cards': 4, 'red_cards': 0, 'corner_kicks': 0, 'free_kicks': 8}
    },
    135: {
        "home team": {'goals': 0, 'shots': 4, 'fouls': 3, 'yellow_cards': 0, 'red_cards': 0, 'corner_kicks': 0, 'free_kicks': 0},
        "away team": {'goals': 0, 'shots': 3, 'fouls': 5, 'yellow_cards': 1, 'red_cards': 0, 'corner_kicks': 0, 'free_kicks': 2}
    },
    138: {
        "home team": {'goals': 0, 'shots': 0, 'fouls': 8, 'yellow_cards': 1, 'red_cards': 0, 'corner_kicks': 3, 'offsides': 1},
        "away team": {'goals': 0, 'shots': 0, 'fouls': 3, 'yellow_cards': 2, 'red_cards': 0, 'corner_kicks': 3, 'offsides': 2}
    },
    139: {
        "home team": """Goals: 0
                        Shots: 0
                        Fouls: 0
                        Yellow Cards: 0
                        Red Cards: 0
                        Corner Kicks: 0
                        Free Kicks: 0
                        Offsides: 0
                        Events: {}""",
        "away team": """Goals: 0
                        Shots: 0
                        Fouls: 0
                        Yellow Cards: 0
                        Red Cards: 0
                        Corner Kicks: 0
                        Free Kicks: 0
                        Offsides: 0
                        Events: {'goal! home team 1-0 away team. 12 mins': ['Goal! Home Team 1-0 Away Team. 12 mins: GOAL_HOME_TEAM'], 'shot saved! away team. 15 mins': ['Shot saved! Away Team. 15 mins: SHOT_SAVED_AWAY_TEAM'], 'foul by home team. 18 mins': ['Foul by Home Team. 18 mins: FOUL_HOME_TEAM'], 'yellow card for home team player. 20 mins': ['Yellow Card for Home Team player. 20 mins: YELLOW_CARD_HOME_TEAM'], 'penalty for away team. 25 mins': ['Penalty for Away Team. 25 mins: PENALTY_AWAY_TEAM'], 'foul by away team. 30 mins': ['Foul by Away Team. 30 mins: FOUL_AWAY_TEAM'], 'offside by home team player. 35 mins': ['Offside by Home Team player. 35 mins: OFFSIDE_HOME_TEAM'], 'shot on target! away team. 40 mins': ['Shot on target! Away Team. 40 mins: SHOT_ON_TARGET_AWAY_TEAM'], 'foul by home team. 45 mins': ['Foul by Home Team. 45 mins: FOUL_HOME_TEAM'], 'yellow card for away team player. 45+1 mins': ['Yellow Card for Away Team player. 45+1 mins: YELLOW_CARD_AWAY_TEAM'], 'red card for home team player. 45+3 mins': ['Red Card for Home Team player. 45+3 mins: RED_CARD_HOME_TEAM'], 'corner kick for home team. 48 mins': ['Corner kick for Home Team. 48 mins: CORNER_KICK_HOME_TEAM'], 'corner kick for away team. 50 mins': ['Corner kick for Away Team. 50 mins: CORNER_KICK_AWAY_TEAM'], 'corner kick for home team. 55 mins': ['Corner kick for Home Team. 55 mins: CORNER_KICK_HOME_TEAM'], 'free kick for away team. 60 mins': ['Free kick for Away Team. 60 mins: FREE_KICK_AWAY_TEAM'], 'shot blocked! home team. 65 mins': ['Shot blocked! Home Team. 65 mins: SHOT_BLOCKED_HOME_TEAM'], 'foul by away team. 70 mins': ['Foul by Away Team. 70 mins: FOUL_AWAY_TEAM'], 'goal! away team 1-1 home team. 75 mins': ['Goal! Away Team 1-1 Home Team. 75 mins: GOAL_AWAY_TEAM'], 'shot missed! home team. 80 mins': ['Shot missed! Home Team. 80 mins: SHOT_MISSED_HOME_TEAM'], 'handball by away team player. 85 mins': ['Handball by Away Team player. 85 mins: HANDBALL_AWAY_TEAM'], 'foul by home team. 90 mins': ['Foul by Home Team. 90 mins: FOUL_HOME_TEAM']}"""
    },
    140: {'Home Team': {'Goals': 4, 'Shots': 17, 'Fouls': 16, 'Yellow Cards': 2, 'Red Cards': 5, 'Corner Kicks': 4, 'Free Kicks': 3, 'Offsides': 0}, 'Away Team': {'Goals': 3, 'Shots': 18, 'Fouls': 19, 'Yellow Cards': 2, 'Red Cards': 7, 'Corner Kicks': 5, 'Free Kicks': 5, 'Offsides': 2}},
    143: {
        "home team": {'goals': 0, 'shots': 0, 'fouls': 0, 'yellow_cards': 0, 'red_cards': 0, 'corner_kicks': 0, 'offsides': 0},
        "away team": {'goals': 0, 'shots': 0, 'fouls': 0, 'yellow_cards': 0, 'red_cards': 0, 'corner_kicks': 0, 'offsides': 0}
    },
    149: {
        "home team": {'goals': 0, 'shots': 7, 'fouls': 3, 'yellow_cards': 2, 'red_cards': 1, 'corner_kicks': 4, 'free_kicks': 14},
        "away team": {'goals': 0, 'shots': 0, 'fouls': 3, 'yellow_cards': 2, 'red_cards': 0, 'corner_kicks': 1, 'free_kicks': 9}
    },
    159: {'Home Team': {'goals': 0, 'shots': 0, 'fouls': 0, 'yellow_cards': 0, 'red_cards': 0, 'corner_kicks': 7, 'free_kicks': 13, 'offside': 1}, 'Away Team': {'goals': 0, 'shots': 0, 'fouls': 0, 'yellow_cards': 1, 'red_cards': 0, 'corner_kicks': 2, 'free_kicks': 11, 'offside': 0}}
}

In [38]:
map_2_3 = []
prompt_3_list = []
for i in range( len(predict_2_list) ):
    if i in excuta:
        context = excuta[i]
        map_2_3.append( i )
        prompt_3 = stage_3_instruction.format( INSTRUCTION = general_instruction,  Text = context )
        prompt_3_list.append( [prompt_3] )

prompt_3_path = os.path.join(_ROOT_PATH, "prompt_3_list.pickle" )
save_data( prompt_3_list, prompt_3_path)
save_data( map_2_3, prompt_3_path.replace( "prompt_3_list", "prompt_3_map_2"))

In [39]:
kwargs = {}
prompt_list_from_path = prompt_3_path  # prompt_2_merge_path
kwargs["local_or_remote"] = "local"
kwargs["server_or_reader"] = "reader"
kwargs["model_name"] = "Mistral-7B-Instruct-v0.2"
kwargs["gpu_list"] = "4,5"
kwargs["local_engine"] = "vllm"
if prompt_list_from_path != None:
    kwargs["prompt_list_from_path"] = prompt_list_from_path
    kwargs["predict_list_to_path"] = prompt_list_from_path.replace( "prompt_3_list.pickle", "predict_3_list.pickle")
    kwargs["sample_little"] = None
inferencer = Inferencer( kwargs )

Enginevllm exists, run with vllm
WARNING 12-10 11:27:48 config.py:748] Casting torch.bfloat16 to torch.float16.
INFO 12-10 11:27:48 llm_engine.py:75] Initializing an LLM engine (v0.4.0) with config: model='/home/jiangpeiwen2/jiangpeiwen2/workspace/LLMs/Mistral-7B-Instruct-v0.2', tokenizer='/home/jiangpeiwen2/jiangpeiwen2/workspace/LLMs/Mistral-7B-Instruct-v0.2', tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.float16, max_seq_len=8192, download_dir=None, load_format=auto, tensor_parallel_size=1, disable_custom_all_reduce=True, quantization=None, enforce_eager=False, kv_cache_dtype=auto, device_config=cuda, seed=32)
WARNING 12-10 11:27:48 config.py:748] Casting torch.bfloat16 to torch.float16.
INFO 12-10 11:27:48 llm_engine.py:75] Initializing an LLM engine (v0.4.0) with config: model='/home/jiangpeiwen2/jiangpeiwen2/workspace/LLMs/Mistral-7B-Instruct-v0.2', tokenizer='/home/jiangpeiwen2/jiangpeiwen2/workspace/LLMs/Mistral-7B-Instruct-v0.